# 第 4 章：分离数据与指令

- [课程](#lesson)
- [练习](#exercises)
- [示例演练场](#example-playground)

## 设置

运行以下设置单元格以加载您的 API 密钥并建立`get_completion`辅助函数。

In [ ]:
%pip install anthropic

# 导入 Python 内置的正则表达式库
import re
import anthropic

# 从 IPython 存储中检索 API_KEY 和 MODEL_NAME 变量
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

def get_completion(prompt: str, system_prompt=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt}
        ]
    )
    return message.content[0].text

---

## 课程

通常，我们不希望每次都编写完整的提示，而是希望使用**提示模板，这些模板可以在提交给 Claude 之前，通过额外的输入数据进行修改**。如果您希望 Claude 每次都执行相同的任务，但用于该任务的数据每次可能不同，那么这种方法就会派上用场。

幸运的是，我们可以通过**将提示的固定骨架与可变的用户输入分离，然后将用户输入替换到提示中**，再将完整的提示发送给 Claude，从而轻松实现这一点。

下面，我们将逐步讲解如何编写一个可替换的提示模板，以及如何将用户输入替换进去。

### 示例

在这个第一个示例中，我们要求 Claude 扮演一个动物叫声生成器。请注意，提交给 Claude 的完整提示只是用输入（在本例中是“Cow”）替换了`PROMPT_TEMPLATE`。请注意，当我们打印出完整的提示时，单词“Cow”通过 f-string 替换了`ANIMAL`占位符。

**注意：** 实际上，您不必将占位符变量命名为任何特定的名称。在这个示例中，我们将其命名为`ANIMAL`，但同样可以将其命名为`CREATURE`或`A`（尽管通常最好让您的变量名具体且相关，这样即使没有替换，您的提示模板也易于理解，仅仅是为了用户可读性）。只需确保您为变量命名的名称与您在提示模板 f-string 中使用的名称一致即可。

In [ ]:
# 可变内容
ANIMAL = "奶牛"

# 带有可变内容占位符的提示模板
PROMPT = f"我将告诉你一种动物的名字。请回复该动物发出的声音。{ANIMAL}"

# 打印 Claude 的响应
print("--------------------------- 带有变量替换的完整提示 ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude 的响应 -------------------------------------")
print(get_completion(PROMPT))

我们为什么要这样分离和替换输入呢？嗯，**提示模板简化了重复性任务**。假设您构建了一个提示结构，允许第三方用户向提示提交内容（在本例中是他们想要生成叫声的动物）。这些第三方用户不必编写甚至不必看到完整的提示。他们所要做的就是填写变量。

我们在这里使用变量和 f-string 进行替换，但您也可以使用`format()`方法来完成。

**注意：** 提示模板可以包含任意数量的变量！

在引入这样的替换变量时，非常重要的是要**确保 Claude 知道变量的开始和结束位置**（相对于指令或任务描述）。让我们看一个指令和替换变量之间没有分离的例子。

对于我们人类的眼睛来说，在下面的提示模板中，变量的开始和结束位置非常清楚。然而，在完全替换后的提示中，这种界限变得模糊不清。

In [ ]:
# 可变内容
EMAIL = "明天早上 6 点出现，因为我是 CEO，我说了算。"

# 带有可变内容占位符的提示模板
PROMPT = f"嘿 Claude。{EMAIL} <----- 让这封电子邮件更礼貌，但不要改变其他任何内容。"

# 打印 Claude 的响应
print("--------------------------- 带有变量替换的完整提示 ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude 的响应 -------------------------------------")
print(get_completion(PROMPT))

在这里，**Claude 认为“Yo Claude”是它应该重写的电子邮件的一部分**！你可以从它以“Dear Claude”开始重写中看出来。对于人类的眼睛来说，这很清楚，尤其是在提示模板中电子邮件的开始和结束位置，但在替换后的提示中，这种界限变得模糊不清。

我们如何解决这个问题？**将输入内容用 XML 标签包裹起来**！我们在下面这样做了，正如你所看到的，输出中不再有“Dear Claude”了。

[XML 标签](https://docs.anthropic.com/claude/docs/use-xml-tags)是像`<tag></tag>`这样的尖括号标签。它们成对出现，由一个开始标签（例如`<tag>`）和一个带有斜杠的结束标签（例如`</tag>`）组成。XML 标签用于包裹内容，像这样：`<tag>内容</tag>`。

**注意：** 尽管 Claude 可以识别并处理各种分隔符和定界符，但我们建议您**专门使用 XML 标签作为 Claude 的分隔符**，因为 Claude 经过专门训练，能够将 XML 标签识别为一种提示组织机制。除了函数调用之外，**没有经过特殊训练的 XML 标签可以最大程度地提升您的性能**。我们特意让 Claude 以这种方式变得非常灵活和可定制。


In [ ]:
# 可变内容
EMAIL = "明天早上 6 点出现，因为我是 CEO，我说了算。"

# 带有可变内容占位符的提示模板
PROMPT = f"嘿 Claude。 <email>{EMAIL}</email> <----- 让这封电子邮件更礼貌，但不要改变其他任何内容。"

# 打印 Claude 的响应
print("--------------------------- 带有变量替换的完整提示 ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude 的响应 -------------------------------------")
print(get_completion(PROMPT))

让我们看另一个 XML 标签如何帮助我们的例子。

在下面的提示中，**Claude 错误地解释了提示的哪一部分是指令，哪一部分是输入**。由于格式问题，它错误地将“`Each is about an animal, like rabbits`”视为列表的一部分，而用户（填写`SENTENCES`变量的人）大概并不希望这样。

In [ ]:
# 可变内容
SENTENCES = """- 我喜欢奶牛的声音
- 这句话是关于蜘蛛的
- 这句话看起来是关于狗的，但实际上是关于猪的"""

# 带有可变内容占位符的提示模板
PROMPT = f"""下面是一个句子列表。告诉我列表中第二个项目是什么。

- 每个都与一种动物有关，比如兔子。
{SENTENCES}"""

# 打印 Claude 的响应
print("--------------------------- 带有变量替换的完整提示 ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude 的响应 -------------------------------------")
print(get_completion(PROMPT))

要解决这个问题，我们只需**将用户输入的句子用 XML 标签包裹起来**。这向 Claude 表明了输入数据的开始和结束位置，尽管在“`每个都与一种动物有关，比如兔子。`”之前存在误导性的连字符。

In [ ]:
# 可变内容
SENTENCES = """- 我喜欢奶牛的声音
- 这句话是关于蜘蛛的
- 这句话看起来是关于狗的，但实际上是关于猪的"""

# 带有可变内容占位符的提示模板
PROMPT = f"""下面是一个句子列表。告诉我列表中第二个项目是什么。

- 每个都与一种动物有关，比如兔子。
<sentences>
{SENTENCES}
</sentences>"""

# 打印 Claude 的响应
print("--------------------------- 带有变量替换的完整提示 ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude 的响应 -------------------------------------")
print(get_completion(PROMPT))

**注意：** 在“每句话都关于一种动物”提示的错误版本中，我们不得不包含连字符，以便让 Claude 以我们希望的方式错误地响应，以用于本例。这是一个关于提示的重要教训：**小细节很重要**！**仔细检查您的提示是否存在拼写错误和语法错误**总是值得的。Claude 对模式很敏感（在早期，即微调之前，它是一个原始的文本预测工具），当您犯错时，它更容易犯错；当您听起来聪明时，它会更聪明；当您听起来傻气时，它会更傻气，等等。

如果您想在不更改上述任何内容的情况下尝试课程提示，请一直滚动到课程笔记本的底部，访问[**示例演练场**](#example-playground)。

---

## 练习
- [练习 4.1 - 俳句主题](#exercise-41---haiku-topic)
- [练习 4.2 - 带有拼写错误的狗问题](#exercise-42---dog-question-with-typos)
- [练习 4.3 - 狗问题 第二部分](#exercise-42---dog-question-part-2)

### 练习 4.1 - 俳句主题
修改 `PROMPT`，使其成为一个模板，能够接收名为 `TOPIC` 的变量，并输出关于该主题的俳句。本练习旨在测试你对使用 f-string 进行变量模板化结构的理解。

In [ ]:
# 变量内容
TOPIC = "Pigs"

# 带有变量内容占位符的提示模板
PROMPT = f""

# 获取 Claude 的响应
response = get_completion(PROMPT)

# 评估练习正确性的函数
def grade_exercise(text):
    return bool(re.search("pigs", text.lower()) and re.search("haiku", text.lower()))

# 打印 Claude 的响应
print("--------------------------- 包含变量替换的完整提示 ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude 的响应 -------------------------------------")
print(response)
print("\n------------------------------------------ 评分 ------------------------------------------")
print("此练习已正确解决：", grade_exercise(response))

❓ 如果你需要提示，运行下面的单元格！

In [ ]:
from hints import exercise_4_1_hint; print(exercise_4_1_hint)

### 练习 4.2 - 带有错别字的狗问题
通过添加 XML 标签来修复 `PROMPT`，以便 Claude 能够给出正确的答案。

尽量不要改变提示中的其他任何内容。凌乱且充满错误的写法是故意的，这样你就可以看到 Claude 对此类错误作何反应。

In [ ]:
# 变量内容
QUESTION = "ar cn brown?"

# 带有变量内容占位符的提示模板
PROMPT = f"嗨，是我，我有一个关于狗的问题 jkaerjv {QUESTION} jklmvca 谢谢，帮我很多，很快很快回答，简短简短，谢谢"

# 获取 Claude 的响应
response = get_completion(PROMPT)

# 评估练习正确性的函数
def grade_exercise(text):
    return bool(re.search("棕色", text.lower()))

# 打印 Claude 的响应
print("--------------------------- 包含变量替换的完整提示 ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude 的响应 -------------------------------------")
print(response)
print("\n------------------------------------------ 评分 ------------------------------------------")
print("此练习已正确解决：", grade_exercise(response))

❓ 如果你需要提示，运行下面的单元格！

In [ ]:
from hints import exercise_4_2_hint; print(exercise_4_2_hint)

### 练习 4.3 - 狗问题 第二部分
**不**添加 XML 标签来修复 `PROMPT`。相反，只从提示中删除一两个词。

和上面的练习一样，尽量不要改变提示中的其他任何内容。这将向你展示 Claude 能够解析和理解哪种类型的语言。

In [ ]:
# 变量内容
QUESTION = "ar cn brown?"

# 带有变量内容占位符的提示模板
PROMPT = f"嗨，是我，我有一个关于狗的问题 jkaerjv {QUESTION} jklmvca 谢谢，帮我很多，很快很快回答，简短简短，谢谢"

# 获取 Claude 的响应
response = get_completion(PROMPT)

# 评估练习正确性的函数
def grade_exercise(text):
    return bool(re.search("棕色", text.lower()))

# 打印 Claude 的响应
print("--------------------------- 包含变量替换的完整提示 ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude 的响应 -------------------------------------")
print(response)
print("\n------------------------------------------ 评分 ------------------------------------------")
print("此练习已正确解决：", grade_exercise(response))

❓ 如果你需要提示，运行下面的单元格！

In [ ]:
from hints import exercise_4_3_hint; print(exercise_4_3_hint)

### 恭喜！

如果你已经解决了到目前为止的所有练习，你就可以进入下一章了。祝你提示愉快！

---

# Chapter 4: Separating Data and Instructions

- [Lesson](#lesson)
- [Exercises](#exercises)
- [Example Playground](#example-playground)

## Setup

Run the following setup cell to load your API key and establish the `get_completion` helper function.

In [ ]:
%pip install anthropic

# Import python's built-in regular expression library
import re
import anthropic

# Retrieve the API_KEY & MODEL_NAME variables from the IPython store
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

def get_completion(prompt: str, system_prompt=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt}
        ]
    )
    return message.content[0].text

---

## Lesson

Oftentimes, we don't want to write full prompts, but instead want **prompt templates that can be modified later with additional input data before submitting to Claude**. This might come in handy if you want Claude to do the same thing every time, but the data that Claude uses for its task might be different each time. 

Luckily, we can do this pretty easily by **separating the fixed skeleton of the prompt from variable user input, then substituting the user input into the prompt** before sending the full prompt to Claude. 

Below, we'll walk step by step through how to write a substitutable prompt template, as well as how to substitute in user input.

### Examples

In this first example, we're asking Claude to act as an animal noise generator. Notice that the full prompt submitted to Claude is just the `PROMPT_TEMPLATE` substituted with the input (in this case, "Cow"). Notice that the word "Cow" replaces the `ANIMAL` placeholder via an f-string when we print out the full prompt.

**Note:** You don't have to call your placeholder variable anything in particular in practice. We called it `ANIMAL` in this example, but just as easily, we could have called it `CREATURE` or `A` (although it's generally good to have your variable names be specific and relevant so that your prompt template is easy to understand even without the substitution, just for user parseability). Just make sure that whatever you name your variable is what you use for the prompt template f-string.

In [ ]:
# Variable content
ANIMAL = "Cow"

# Prompt template with a placeholder for the variable content
PROMPT = f"I will tell you the name of an animal. Please respond with the noise that animal makes. {ANIMAL}"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

Why would we want to separate and substitute inputs like this? Well, **prompt templates simplify repetitive tasks**. Let's say you build a prompt structure that invites third party users to submit content to the prompt (in this case the animal whose sound they want to generate). These third party users don't have to write or even see the full prompt. All they have to do is fill in variables.

We do this substitution here using variables and f-strings, but you can also do it with the format() method.

**Note:** Prompt templates can have as many variables as desired!

When introducing substitution variables like this, it is very important to **make sure Claude knows where variables start and end** (vs. instructions or task descriptions). Let's look at an example where there is no separation between the instructions and the substitution variable.

To our human eyes, it is very clear where the variable begins and ends in the prompt template below. However, in the fully substituted prompt, that delineation becomes unclear.

In [ ]:
# Variable content
EMAIL = "Show up at 6am tomorrow because I'm the CEO and I say so."

# Prompt template with a placeholder for the variable content
PROMPT = f"Yo Claude. {EMAIL} <----- Make this email more polite but don't change anything else about it."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

Here, **Claude thinks "Yo Claude" is part of the email it's supposed to rewrite**! You can tell because it begins its rewrite with "Dear Claude". To the human eye, it's clear, particularly in the prompt template where the email begins and ends, but it becomes much less clear in the prompt after substitution.

How do we solve this? **Wrap the input in XML tags**! We did this below, and as you can see, there's no more "Dear Claude" in the output.

[XML tags](https://docs.anthropic.com/claude/docs/use-xml-tags) are angle-bracket tags like `<tag></tag>`. They come in pairs and consist of an opening tag, such as `<tag>`, and a closing tag marked by a `/`, such as `</tag>`. XML tags are used to wrap around content, like this: `<tag>content</tag>`.

**Note:** While Claude can recognize and work with a wide range of separators and delimeters, we recommend that you **use specifically XML tags as separators** for Claude, as Claude was trained specifically to recognize XML tags as a prompt organizing mechanism. Outside of function calling, **there are no special sauce XML tags that Claude has been trained on that you should use to maximally boost your performance**. We have purposefully made Claude very malleable and customizable this way.

In [ ]:
# Variable content
EMAIL = "Show up at 6am tomorrow because I'm the CEO and I say so."

# Prompt template with a placeholder for the variable content
PROMPT = f"Yo Claude. <email>{EMAIL}</email> <----- Make this email more polite but don't change anything else about it."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

Let's see another example of how XML tags can help us. 

In the following prompt, **Claude incorrectly interprets what part of the prompt is the instruction vs. the input**. It incorrectly considers `Each is about an animal, like rabbits` to be part of the list due to the formatting, when the user (the one filling out the `SENTENCES` variable) presumably did not want that.

In [ ]:
# Variable content
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""

# Prompt template with a placeholder for the variable content
PROMPT = f"""Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
{SENTENCES}"""

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

To fix this, we just need to **surround the user input sentences in XML tags**. This shows Claude where the input data begins and ends despite the misleading hyphen before `Each is about an animal, like rabbits.`

In [ ]:
# Variable content
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""

# Prompt template with a placeholder for the variable content
PROMPT = f""" Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
<sentences>
{SENTENCES}
</sentences>"""

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

**Note:** In the incorrect version of the "Each is about an animal" prompt, we had to include the hyphen to get Claude to respond incorrectly in the way we wanted to for this example. This is an important lesson about prompting: **small details matter**! It's always worth it to **scrub your prompts for typos and grammatical errors**. Claude is sensitive to patterns (in its early years, before finetuning, it was a raw text-prediction tool), and it's more likely to make mistakes when you make mistakes, smarter when you sound smart, sillier when you sound silly, and so on.

If you would like to experiment with the lesson prompts without changing any content above, scroll all the way to the bottom of the lesson notebook to visit the [**Example Playground**](#example-playground).

---

## Exercises
- [Exercise 4.1 - Haiku Topic](#exercise-41---haiku-topic)
- [Exercise 4.2 - Dog Question with Typos](#exercise-42---dog-question-with-typos)
- [Exercise 4.3 - Dog Question Part 2](#exercise-42---dog-question-part-2)

### Exercise 4.1 - Haiku Topic
Modify the `PROMPT` so that it's a template that will take in a variable called `TOPIC` and output a haiku about the topic. This exercise is just meant to test your understanding of the variable templating structure with f-strings.

In [ ]:
# Variable content
TOPIC = "Pigs"

# Prompt template with a placeholder for the variable content
PROMPT = f""

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("pigs", text.lower()) and re.search("haiku", text.lower()))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ If you want a hint, run the cell below!

In [ ]:
from hints import exercise_4_1_hint; print(exercise_4_1_hint)

### Exercise 4.2 - Dog Question with Typos
Fix the `PROMPT` by adding XML tags so that Claude produces the right answer. 

Try not to change anything else about the prompt. The messy and mistake-ridden writing is intentional, so you can see how Claude reacts to such mistakes.

In [ ]:
# Variable content
QUESTION = "ar cn brown?"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hia its me i have a q about dogs jkaerjv {QUESTION} jklmvca tx it help me muhch much atx fst fst answer short short tx"

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("brown", text.lower()))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ If you want a hint, run the cell below!

In [ ]:
from hints import exercise_4_2_hint; print(exercise_4_2_hint)

### Exercise 4.3 - Dog Question Part 2
Fix the `PROMPT` **WITHOUT** adding XML tags. Instead, remove only one or two words from the prompt.

Just as with the above exercises, try not to change anything else about the prompt. This will show you what kind of language Claude can parse and understand.

In [ ]:
# Variable content
QUESTION = "ar cn brown?"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hia its me i have a q about dogs jkaerjv {QUESTION} jklmvca tx it help me muhch much atx fst fst answer short short tx"

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("brown", text.lower()))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ If you want a hint, run the cell below!

In [ ]:
from hints import exercise_4_3_hint; print(exercise_4_3_hint)

### Congrats!

If you've solved all exercises up until this point, you're ready to move to the next chapter. Happy prompting!

---

## Example Playground

This is an area for you to experiment freely with the prompt examples shown in this lesson and tweak prompts to see how it may affect Claude's responses.

In [ ]:
# Variable content
ANIMAL = "Cow"

# Prompt template with a placeholder for the variable content
PROMPT = f"I will tell you the name of an animal. Please respond with the noise that animal makes. {ANIMAL}"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Variable content
EMAIL = "Show up at 6am tomorrow because I'm the CEO and I say so."

# Prompt template with a placeholder for the variable content
PROMPT = f"Yo Claude. {EMAIL} <----- Make this email more polite but don't change anything else about it."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Variable content
EMAIL = "Show up at 6am tomorrow because I'm the CEO and I say so."

# Prompt template with a placeholder for the variable content
PROMPT = f"Yo Claude. <email>{EMAIL}</email> <----- Make this email more polite but don't change anything else about it."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Variable content
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""

# Prompt template with a placeholder for the variable content
PROMPT = f"""Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
{SENTENCES}"""

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Variable content
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""

# Prompt template with a placeholder for the variable content
PROMPT = f""" Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
<sentences>
{SENTENCES}
</sentences>"""

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))